# Loading partially-preprocessed data from docdb database

In [1]:
import sys

In [2]:
sys.path.append('/auto/k1/robertg/code/docdb/')
sys.path.append('/auto/k1/mark/Code/')

In [3]:
import os
import docdb
import numpy as np
import vm_tools as vmt
from scipy.io import loadmat

In [4]:
# 1a and 1b refer to two different tasks
sub_list = ['AH', 'DS', 'ML', 'WH']
exp_names = dict((sub, dict(ses1a='Sub%s_BVP_Ses1a'%sub,
                            ses1b='Sub%s_BVP_Ses1b'%sub)) for sub in sub_list)

In [5]:
exp_names

{'AH': {'ses1a': 'SubAH_BVP_Ses1a', 'ses1b': 'SubAH_BVP_Ses1b'},
 'DS': {'ses1a': 'SubDS_BVP_Ses1a', 'ses1b': 'SubDS_BVP_Ses1b'},
 'ML': {'ses1a': 'SubML_BVP_Ses1a', 'ses1b': 'SubML_BVP_Ses1b'},
 'WH': {'ses1a': 'SubWH_BVP_Ses1a', 'ses1b': 'SubWH_BVP_Ses1b'}}

In [6]:
exp_names = {'AH': ['20100824AH-sg', '20100829AH-sg', '20110116AH-sg'],
            'AN': ['20130607AN', '20140221AN', '20140223AN'],
            'AV': ['20101002AV-sg', '20101009AV-sg', '20101022AV-sg'],
            'BG': ['20140217BG', '20140301BG-sg', '20140303BG-sg'],
            'DS': ['20140301DS-sg', '20140307DS-sg', '20140309DS-sg'],
            'JG': ['20110321JG-sg', '20110615JG-sg', '20110621JG-sg'],
            'ML': ['20130117ML-sg', '20130307ML-sg', '20130312ML-sg'],
            'NB': ['20110130NB-sg', '20110320NB-sg', '20110605NB-sg'],
            'SN': ['20100714SN-sg', '20100828SN-sg', '20100830SN-sg'],
            'TC': ['20101001TC-sg', '20110219TC-sg', '20110522TC-sg'],
            'WH': ['20120804WH-sg', '20120814WH-sg', '20121128WH-sg'],
            }

In [7]:
# This will vary depending on who you are getting data from
# dbname = 'docdb-ml'
dbname = 'docdb-nb'
docdbi = docdb.getclient(dbname=dbname)

Contacting
           host : http://meth:5984
       database : docdb-nb
Missing input xfm1 to action ConcatTransforms (_id=49c0a9f2ac19f6deff030f80e98532e63f2d0edeb9a4fb4230ad8993fe52854c0660c52323275e88f83a56342297e13d880592e6848edbb9cbb9224d93211697);
	should be from transforms output from action with _id=2fff993b60b5f67bb87f2d2afd1f2d1a82350b85aa56ea3748783d3365b28558053608a84ba498bbadad8b614609caf377968d672b6b8384d86277a585ea7937
Missing input image to action TemporalMean (_id=d7181f30c631a4e7fad17e9a1afa7b5aa37238a22332474a61cac142011795eaac3a5f473bc31de0bdbba8848946786515160b10151f62bb55b5138fc6a9f804);
	should be from image output from action with _id=2fff993b60b5f67bb87f2d2afd1f2d1a82350b85aa56ea3748783d3365b28558053608a84ba498bbadad8b614609caf377968d672b6b8384d86277a585ea7937
Missing input image to action DetrendMedian (_id=deb456908a449be6fa2f6394112d61651d711ec8d00cb702bbef02573323a055e34b8863817908aa2fda6c2653b7676d957f94664342c87545cc62fe66a3e77c);
	should be from image o

In [8]:
def check_ah(runs):
    """Bullshit special case bullshit"""
    dep_act = [r.get_generating_action().get_all_dependencies() for r in runs]
    dep_exp = [[ax.experiment_name for ax in a] for a in dep_act]
    return [r for chk, r in zip(dep_exp, runs) if not 'SubAH_BVP_Ses1a' in chk]

In [9]:
def load_exp_data(experiment_name, clip=(10,310)):
    """Load data for movies (Blender movies / natural movies).     
    """
    runs = docdbi.query(experiment_name=experiment_name, 
                        generated_by_name='ApplyTransform')
    # Some bullshit
    if experiment_name=='SubAH_BVP_Ses1b':
        runs = check_ah(runs)
    if len(runs)==6:
        # For second and third sessions, all runs (1-7) will have 
        # ApplyTransform applied to them; thus this won't be necessary
        r1 = docdbi.query(experiment_name=experiment_name, 
                      block_number=0, 
                      generated_by_name='MotionCorrectFSL')
        runs = r1+runs
    docs = sorted(runs, key=lambda x: x.block_number)
    tdata = np.vstack([d.get_data()[clip[0]:clip[1]] for t, d in enumerate(docs) if t in (0,2,4,6)])
    vdata = np.vstack([d.get_data()[clip[0]:clip[1]] for v, d in enumerate(docs) if v in (1,3,5)])
    return tdata, vdata

# Used to check on output for blender experiments

only works if load_exp_data is set to return `docs` variable.

    docdbi.is_verbose = False
    for sub in sub_list:
        for ses in ['ses1a','ses1b']:
            print('Subject %s, %s'%(sub,ses))
            print len(load_exp_data(exp_names[sub][ses]))

In [13]:
for exp_name in exp_names['JG']:
    print exp_name,
    tdata, vdata = load_exp_data(exp_names['AH'][0])
    np.savez('/media/robertg/BOB_SAGET/datasets/'+exp_name+'.npz', tdata=tdata, vdata=vdata)

20110321JG-sg returning OBJECTS
"experiment_name" query returns 42 items; elapsed time is 0.040
"generated_by_name" query returns 6 items; elapsed time is 0.041


returning OBJECTS
"block_number" query returns 678 items; elapsed time is 0.025
"experiment_name" query returns 5 items; elapsed time is 0.028
"generated_by_name" query returns 1 items; elapsed time is 0.028


20110615JG-sg returning OBJECTS
"experiment_name" query returns 42 items; elapsed time is 0.038
"generated_by_name" query returns 6 items; elapsed time is 0.038


returning OBJECTS
"block_number" query returns 678 items; elapsed time is 0.025
"experiment_name" query returns 5 items; elapsed time is 0.028
"generated_by_name" query returns 1 items; elapsed time is 0.028


20110621JG-sg returning OBJECTS
"experiment_name" query returns 42 items; elapsed time is 0.029
"generated_by_name" query returns 6 items; elapsed time is 0.029


returning OBJECTS
"block_number" query returns 678 items; elapsed time is 0.025
"experiment

In [11]:
print tdata.shape
print vdata.shape

(1200, 32, 100, 100)
(900, 32, 100, 100)


In [48]:
# For Blender movies
order_file_blender = "/auto/k1/mark/Projects-GLab/BVP_StimPres/BVPpilot1_Movies/ValSeq.mat"
seq_blender = loadmat(order_file_blender)['seq'].flatten()

In [49]:
order_file_movies = os.path.join(os.path.split(os.path.split(docdb.__file__)[0])[0], "pipelines/ClipOrderOneMin_3by10.mat")
seq_movies = loadmat(order_file_movies)['seq'].flatten()

In [50]:
print seq_movies
print seq_blender

[2 1 3 2 3 1 3 2 1 3 1 2 3 1 2 1 2 3 2 1 3 3 2 1 1 2 3 1 2 3]
[3 2 1 2 1 3 1 2 3 2 2 1 1 3 3 2 2 3 3 1 1 3 1 2 1 3 2 3 2 1]


In [51]:
# To sort valdation data by repeat:
rvdata = vmt.utils.timecourse2repeats(vdata, seq_blender, 30, collapse_fun=None)

In [52]:
rvdata.shape

(10, 90, 300000)

In [ ]:
# You will want to mask this (obviously).